# Spicata_Clean_and_Prep
### Clustering User Data According to Proximity in Time
The commented out code below was an attempt at trying to define a "hike" by proximity in time, but I found that the exercise wasn't very successful at defining a cluster, the code took too long and the time_diff category was plenty good for EDA around proximity in time.

In [35]:
# sorting data by user then by time_observed_at so "time_diff" operates on a single user in the order
# the observations were made
# NOTE: there may be a reason to choose "created_at" or "id" instead if "time_observed_at" is suspect
data_sp = data_sp.sort_values(by = ["user_id", "time_observed_at"]).reset_index(drop = True)

In [54]:
# column that calculates time difference between given row and previous row in total seconds
data_sp["time_diff"] = data_sp["time_observed_at"].diff()
data_sp["time_diff"] = data_sp["time_diff"].dt.total_seconds() / 60

For each user's first entry "time_diff", the calculated value compared to previous user's last entry doesn't make good sense. There are some options for what could be imputed there. The mean or median of that user's time differences, for example. But also perhaps 0? I'm starting with -0.001 because it's close to zero and so will act a lot like zero in this dataset if included in summary stats, but can be more easily filtered out of the dataset all together, since all actual "time differences" are zero or positive given that I constructed the set sequentially in time for each user.

In [56]:
users = data_sp["user_id"].unique() # identifies users in set --REQUIRED FOR LATER CODE
imputed_initial_time = -0.001  # change here for different imputation
for user in users:
    min_index = data_sp.index[data_sp["user_id"] == user].min()
    data_sp.loc[min_index, "time_diff"] = imputed_initial_time

In [57]:
# summary stats with -0.001 as initial time_diff included
round(data_sp["time_diff"].describe(),3)

count      70865.000
mean        1978.990
std        18687.089
min           -0.001
25%            1.083
50%            4.950
75%           51.650
max      1089394.000
Name: time_diff, dtype: float64

In [58]:
# summary stats with -0.001 as initial time_diff excluded
round(data_sp[data_sp["time_diff"]!= -0.001]["time_diff"].describe(),3)

count      70815.000
mean        1980.387
std        18693.611
min            0.000
25%            1.083
50%            4.950
75%           51.733
max      1089394.000
Name: time_diff, dtype: float64

The two cells below are attempts to try to cluster user data into "hikes", but the code itself takes a lot of time to run and I am unclear that it is helpful.

In [39]:
# NOTE--THIS CODE ONLY WORKS IF DATA ORGANIZED BY USER and THEN BY EITHER ID/CREATED_AT OR TIME_OBSERVED_AT
# classifies observations into single "hikes" as defined by those proximal in time
# time is chosen by max_gap_min value
'''
data_sp["time_cluster"] = 1 # creates column to fill, also sets initial hike number at 1 
users = data_sp["user_id"].unique() # identifies users in set
max_gap_min = 120 #enter the maximum number of minutes here that would define a "new hike"
for user in users:
    cluster_num = 1 
    min_index = data_sp.index[data_sp["user_id"] == user].min()
    max_index = data_sp.index[data_sp["user_id"] == user].max()
    for i in range(min_index + 1, max_index + 1):  
        if data_sp[data_sp["user_id"] == user]["time_diff"][i] <= max_gap_min:
            data_sp.loc[i, "time_cluster"] = cluster_num  # if next entry is inside of time bound, logged as same cluster
        else: 
            cluster_num += 1
            data_sp.loc[i, "time_cluster"] = cluster_num #
    '''

'\ndata_sp["time_cluster"] = 1 # creates column to fill, also sets initial hike number at 1 \nusers = data_sp["user_id"].unique() # identifies users in set\nmax_gap_min = 120 #enter the maximum number of minutes here that would define a "new hike"\nfor user in users:\n    cluster_num = 1 \n    min_index = data_sp.index[data_sp["user_id"] == user].min()\n    max_index = data_sp.index[data_sp["user_id"] == user].max()\n    for i in range(min_index + 1, max_index + 1):  \n        if data_sp[data_sp["user_id"] == user]["time_diff"][i] <= max_gap_min:\n            data_sp.loc[i, "time_cluster"] = cluster_num  # if next entry is inside of time bound, logged as same cluster\n        else: \n            cluster_num += 1\n            data_sp.loc[i, "time_cluster"] = cluster_num #\n    '

In [40]:
#uses code above to compare different time gaps for clustering a "hike"
#tries to compare spread for different time gaps but takes SOOOOOO long to run and not sure it's useful
# so it's commented out for now
# STILL, there is a question of "what arbitrary cut-off is there for 'proximity in time' as a single 'hike'?
'''
users = data_sp["user_id"].unique() 
spread_hikes_by_time_gap = {}
max_gap_min = [60, 120, 180, 240] 
for time in max_gap_min:
    data_sp["time_cluster"] = 1
    for user in users:
        cluster_num = 1 
        min_index = data_sp.index[data_sp["user_id"] == user].min()
        max_index = data_sp.index[data_sp["user_id"] == user].max()
        for i in range(min_index + 1, max_index + 1):  
            if data_sp[data_sp["user_id"] == user]["time_diff"][i] <= time:
                data_sp.loc[i, "time_cluster"] = cluster_num  
            else: 
                cluster_num += 1
                data_sp.loc[i, "time_cluster"] = cluster_num 
    spread_hikes_by_time_gap[time] = data_sp.groupby(["user_id"]).max()["time_cluster"].describe().values
spread_hikes_by_time_gap
''' 


'\nusers = data_sp["user_id"].unique() \nspread_hikes_by_time_gap = {}\nmax_gap_min = [60, 120, 180, 240] \nfor time in max_gap_min:\n    data_sp["time_cluster"] = 1\n    for user in users:\n        cluster_num = 1 \n        min_index = data_sp.index[data_sp["user_id"] == user].min()\n        max_index = data_sp.index[data_sp["user_id"] == user].max()\n        for i in range(min_index + 1, max_index + 1):  \n            if data_sp[data_sp["user_id"] == user]["time_diff"][i] <= time:\n                data_sp.loc[i, "time_cluster"] = cluster_num  \n            else: \n                cluster_num += 1\n                data_sp.loc[i, "time_cluster"] = cluster_num \n    spread_hikes_by_time_gap[time] = data_sp.groupby(["user_id"]).max()["time_cluster"].describe().values\nspread_hikes_by_time_gap\n'

In [41]:
'''
num_hikes = data_sp.groupby(["user_id"]).nunique()["time_cluster"]
num_obs = data_sp.groupby(["user_id"]).count()["time_cluster"]
plt.figure()
plt.xlabel("Number of Observations")
plt.ylabel("Number of Clusters by Time")
plt.title("As Expected, Strong Positive Correlation Between \nNumber of Observations and Number of 'Hikes'")
plt.scatter(num_obs,num_hikes)
plt.show()
'''

'\nnum_hikes = data_sp.groupby(["user_id"]).nunique()["time_cluster"]\nnum_obs = data_sp.groupby(["user_id"]).count()["time_cluster"]\nplt.figure()\nplt.xlabel("Number of Observations")\nplt.ylabel("Number of Clusters by Time")\nplt.title("As Expected, Strong Positive Correlation Between \nNumber of Observations and Number of \'Hikes\'")\nplt.scatter(num_obs,num_hikes)\nplt.show()\n'

In [42]:
'''
# central tendencies and spread of number of observations per user
data_sp.groupby(["user_id"]).count()["time_cluster"].describe()
'''

'\n# central tendencies and spread of number of observations per user\ndata_sp.groupby(["user_id"]).count()["time_cluster"].describe()\n'

In [43]:
'''
# central tendencies and spread of number of hikes per user
data_sp.groupby(["user_id"]).max()["time_cluster"].describe()
'''

'\n# central tendencies and spread of number of hikes per user\ndata_sp.groupby(["user_id"]).max()["time_cluster"].describe()\n'

In [44]:
'''
#distribution of number of hikes per user
plt.figure()
plt.hist(data_sp.groupby(["user_id"]).max()["time_cluster"].values, bins = 30)
plt.xlabel("Number of 'Hikes' by Time Clustering")
plt.ylabel("Count of Observers")
plt.title(f'As Expected, Clustering at Low End with Outlier Users Posting Much More Often')
plt.show()
'''

'\n#distribution of number of hikes per user\nplt.figure()\nplt.hist(data_sp.groupby(["user_id"]).max()["time_cluster"].values, bins = 30)\nplt.xlabel("Number of \'Hikes\' by Time Clustering")\nplt.ylabel("Count of Observers")\nplt.title(f\'As Expected, Clustering at Low End with Outlier Users Posting Much More Often\')\nplt.show()\n'